In [ ]:
import qiskit

In [ ]:
import json
 
with open('./qiskit_tokens.json', 'r') as file:
    data = json.load(file)
token1 = data['token1']

In [ ]:
qiskit.__version__

'2.0.0'

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# After using the following code, go to your dashboard (https://quantum.ibm.com/)
# and expire your API key (click the refresh button in the API key field)
service = QiskitRuntimeService(channel="ibm_quantum", # ibm_cloud 
                               token=token1) # your IBM Quantum instance name

C:\Users\gandh\AppData\Local\Temp\ipykernel_19840\2063930300.py:5: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, ibm_cloud will be the only valid channel. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService(channel="ibm_quantum", # ibm_cloud


In [ ]:
backend = service.backend(name = 'ibm_brisbane')

In [ ]:
backend.num_qubits

127

In [ ]:
backend.properties

<bound method IBMBackend.properties of <IBMBackend('ibm_brisbane')>>

In [ ]:
backend.status

<bound method IBMBackend.status of <IBMBackend('ibm_brisbane')>>

In [ ]:
backend.check_faulty

In [ ]:
from qiskit import QuantumCircuit
 
# Create a new circuit with two qubits
qc = QuantumCircuit(2)
 
# Add a Hadamard gate to qubit 0
qc.h(0)
 
# Perform a controlled-X gate on qubit 1, controlled by qubit 0
qc.cx(0, 1)
 
# Return a drawing of the circuit using MatPlotLib ("mpl").
# These guides are written by using Jupyter notebooks, which
# display the output of the last line of each cell.
# If you're running this in a script, use `print(qc.draw())` to
# print a text drawing.
qc.draw("mpl")

MissingOptionalLibraryError: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."